In [1]:
import pandas as pd 
import numpy as np 


In [2]:
map_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\map_category_code_non_bank.csv')
data_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\non_bank_financial_report.csv')


In [3]:
map_df.head()

,vi_caption,category_code,en_caption,report_type
0,1. Doanh thu bán hàng và cung cấp dịch vụ,IS_001,Revenue from sales and services rendered,income_statement
1,2. Các khoản giảm trừ doanh thu,IS_002,Deductions from revenue,income_statement
2,3. Doanh thu thuần về bán hàng và cung cấp dịc...,IS_010,Net revenue from sales and services rendered,income_statement
3,4. Giá vốn hàng bán,IS_011,Cost of goods sold,income_statement
4,5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...,IS_020,Gross profit from sales and services rendered,income_statement


In [4]:
data_df.head()

,stock_code,category_code,year,quarter,data
0,MWG,IS_001,2021,0,124141525
1,MWG,IS_001,2022,0,134722248
2,MWG,IS_001,2023,0,119234184
3,MWG,IS_002,2021,0,1183419
4,MWG,IS_002,2022,0,1317471


## Phân tích cấu trúc tài chính 

In [5]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def equity_ratio(equity, total_assets):
    return equity / total_assets if total_assets else None

def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
    return permanent_capital / long_term_assets if long_term_assets else None

def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
    return permanent_capital / fixed_assets if fixed_assets else None

def general_solvency_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def return_on_investment(net_income, total_investment):
    return net_income / total_investment if total_investment else None

def ROIC(NOPAT, invested_capital):
    return NOPAT / invested_capital if invested_capital else None

def return_on_long_term_capital(EBIT, average_long_term_capital):
    return EBIT / average_long_term_capital if average_long_term_capital else None

def basic_earning_power(EBIT, average_total_assets):
    return EBIT / average_total_assets if average_total_assets else None

def debt_to_assets_ratio(total_liabilities, total_assets):
    return total_liabilities / total_assets if total_assets else None

def debt_to_equity_ratio(total_liabilities, equity):
    return total_liabilities / equity if equity else None

def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
    return short_term_liabilities / total_assets if total_assets else None

def interest_coverage_ratio(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
    return long_term_liabilities / equity if equity else None

def short_term_debt_to_equity_ratio(short_term_liabilities, equity):
    return short_term_liabilities / equity if equity else None

# Dictionary to hold functions and corresponding category codes
ratio_functions = {
    'equity_ratio': ['BS_400', 'BS_270'],  # equity, total_assets
    'long_term_asset_self_financing_ratio': [['BS_400', 'BS_330'], 'BS_200'],  # permanent_capital (equity + long_term_liabilities), long_term_assets
    'fixed_asset_self_financing_ratio': [['BS_400', 'BS_330'], ['BS_220','BS_240']],  # permanent_capital (equity + long_term_liabilities), fixed_assets
    'general_solvency_ratio': ['BS_270', 'BS_300'],  #  total_assets, total_liabilities
    'return_on_investment': ['IS_060', 'BS_270'],  # net_income, total_investment (using total assets as example)
    # 'ROIC': ['IS_060', 'BS_200'],  # NOPAT, invested_capital
    # 'return_on_long_term_capital': ['IS_050', 'BS_330'],  # EBIT, long_term_liabilities
    'basic_earning_power': [['IS_050','IS_023'], 'BS_270'],  # EBIT, total_assets
    'debt_to_assets_ratio': ['BS_300', 'BS_270'],  # total_liabilities, total_assets
    'debt_to_equity_ratio': ['BS_300', 'BS_400'],  # total_liabilities, equity
    'short_term_debt_to_assets_ratio': ['BS_310', 'BS_270'],  # short_term_liabilities, total_assets
    'interest_coverage_ratio': [['IS_050','IS_023'], 'IS_023'],  # EBIT, interest_expense
    'long_term_debt_to_equity_ratio': ['BS_330', 'BS_400'],  # long_term_liabilities, equity
    'short_term_debt_to_equity_ratio': ['BS_310', 'BS_400']  # short_term_liabilities, equity
}

# Create a DataFrame to store the results
results_1 = []

# Iterate through pivoted data
for index, row in pivot_df.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        # Fetch input data from the pivot table
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # For cases like permanent capital (sum of multiple values)
                # Sum multiple values if input_name is a list
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_1.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_1 = pd.DataFrame(results_1)

ratios_df_1


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,equity_ratio,0.350236
1,BCM,2021,0,long_term_asset_self_financing_ratio,1.539085
2,BCM,2021,0,fixed_asset_self_financing_ratio,8.668917
3,BCM,2021,0,general_solvency_ratio,1.539021
4,BCM,2021,0,return_on_investment,0.029769
...,...,...,...,...,...
2599,VRE,2024,2,debt_to_equity_ratio,0.310486
2600,VRE,2024,2,short_term_debt_to_assets_ratio,0.105141
2601,VRE,2024,2,interest_coverage_ratio,10.235884
2602,VRE,2024,2,long_term_debt_to_equity_ratio,0.172701


In [6]:
VNM_equity_ratio_q1_2024 = ratios_df_1[
    (ratios_df_1['stock_code'] == 'VNM') & 
    (ratios_df_1['year'] == 2024) & 
    (ratios_df_1['quarter'] == 1) & 
    (ratios_df_1['ratio_code'] == 'equity_ratio')
]['data'].values[0]

print(f"Equity Ratio of VNM in Q1 2024: {VNM_equity_ratio_q1_2024}")

Equity Ratio of VNM in Q1 2024: 0.6943834158870075


In [7]:
# # Financing Ratio (Hệ số tài trợ)
# def equity_ratio(equity, total_assets):
#     return equity / total_assets

# #  Long-Term Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản dài hạn)
# # longterm_assets = BS_200
# def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
#     return permanent_capital / long_term_assets

# #Fixed Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản cố định - TSCĐ)
# # permanent capital = VCSH(400)+ Nợ dài hạn (330)
# # fixed assets = TSCĐ đã và đang đầu tư(220 + 240/242)
# def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
#     return permanent_capital / fixed_assets

# # General Solvency Ratio (Hệ số khả năng thanh toán tổng quát) 
# def general_solvency_ratio(total_liabilities, total_assets):
#     return total_assets / total_liabilities

# # ROI 
# def return_on_investment(net_profit, total_investment):
#     return net_profit / total_investment

# # ROIC (Return on Invested Capital)
# #Net Operating Profit After Taxes (NOPAT) = 60
# def ROIC (NOPAT, invested_capital):
#     return NOPAT / invested_capital

# # Return on Long-Term Capital
# # Earnings Before Interest and Taxes (EBIT) (Lợi nhuận trước thuế và lãi vay) = 50
# def return_on_long_term_capital(EBIT, average_long_term_capital):
#     return EBIT / average_long_term_capital

# # Basic Earning Power (BEP) (Năng lực sinh lời cơ bản)
# def basic_earning_power(EBIT, average_total_assets):
#     return EBIT / average_total_assets

# #Debt-to-Assets Ratio
# def debt_to_assets_ratio(total_liabilities, total_assets):
#     return total_liabilities / total_assets

# # Debt-to-Equity Ratio
# def debt_to_equity_ratio(total_liabilities, equity):
#     return total_liabilities / equity

# # Short-Term Debt-to-Assets Ratio
# def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
#     return short_time_liabilities / total_assets

# #Equity-to-Total Assets Ratio:
# def equity_to_total_assets_ratio(equity, total_assets):
#     return equity / total_assets

# #Interest Coverage Ratio:
# def interest_coverage_ratio(EBIT, interest_expense):
#     return EBIT / interest_expense
# # Long-Term Debt-to-Equity Ratio
# #longterm liabilities = BS_330
# def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
#     return long_term_liabilities / equity

# # Short-Term Debt-to-Total Assets Ratio:
# def short_term_debt_to_total_assets_ratio(short_term_liabilities, total_assets):
#     return short_term_liabilities / total_assets



## Phân tích khả năng thanh toán (LIquidity)

In [8]:
# # Receivables-to-Payables Ratio (Tỷ lệ giữa nợ phải thu và nợ phải trả)
# # accounts_receivable(130+210) / total_liabilities(310+330)
# def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
#     return accounts_receivable / total_liabilities

# #Receivables-to-Total Assets Ratio
# def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
#     return accounts_receivable / total_assets

# #Debt-to-Total Capital Ratio 
# # total_liabilities = BS_300
# # total_capital = BS_440
# def debt_to_total_capital_ratio(total_liabilities, total_capital):
#     return total_liabilities / total_capital

# # the Receivables-to-Sales Ratio (Tỷ lệ giữa nợ phải thu người mua so với tổng số tiền hàng bán ra trong kỳ) 
# # accouts_receivables(131+211) / total_sales (10)
# def receivables_to_sales_ratio(accounts_receivables, total_sales):
#     return accounts_receivables / total_sales

# # Allowance for Doubtful Accounts Ratio (Tỷ lệ giữa dự phòng nợ phải thu khó đòi so với tổng số nợ phải thu người mua)
# # allowance_for_doubtful_accounts(137+219) / accounts_receivables(131+211)
# def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
#     return allowance_for_doubtful_accounts / accounts_receivables

# #Asset-to-Debt Ratio
# def asset_to_debt_ratio(total_assets, total_liabilities):
#     return total_assets / total_liabilities

# # Current ratio (Hệ số thanh khoản ngắn hạn)
# # current_assets = BS_100 - BS_151
# def current_ratio(current_assets, current_liabilities):
#     return current_assets / current_liabilities

# # quick ratio (Hệ số thanh khoản nhanh)
# # current assets - inventory (BS_100 - BS_151 - BS_140) / current liabilities
# def quick_ratio(current_assets, inventory, current_liabilities):
#     return (current_assets - inventory) / current_liabilities

# # Cash Ratio (Hệ số thanh khoản tiền mặt)
# # cash_and_cash_equivalents(BS_110) / current_liabilities
# def cash_ratio(cash_and_cash_equivalents, current_liabilities):
#     return cash_and_cash_equivalents / current_liabilities

# # Current Maturity Coverage Ratio= Cash and Cash Equivalents/ Current Maturing Debt 
# # cash_and_cash_equivalents(BS_110) / current_maturing_debt ( TMBCTC)

# # long_term_debt_coverage_ratio = non_current_assets / long_term_debt
# def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
#     return non_current_assets / non_current_liabilities
# # Debt-to-Equity Ratio = Total Liabilities/ Total Equity
# def debt_to_equity_ratio(total_liabilities, total_equity):
#     return total_liabilities / total_equity

# # Long-Term Debt to Equity Capital Ratio
# # non_current_liabilities = BS_330
# def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
#     return non_current_liabilities / equity

# # Time Interest Earned (TIE) = EBIT(IS_050 + IS_023)/ Interest Expense(IS_023)
# def time_interest_earned(EBIT, interest_expense):
#     return EBIT / interest_expense

# # Debt to Tangible Net Worth Ratio = total_liabilities / equity - intangible_assets (BS_227)
# def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
#     return total_liabilities / (equity - intangible_assets)



 


In [9]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df_2 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
    return accounts_receivable / total_liabilities if total_liabilities else None

def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
    return accounts_receivable / total_assets if total_assets else None

def debt_to_total_capital_ratio(total_liabilities, total_capital):
    return total_liabilities / total_capital if total_capital else None

def receivables_to_sales_ratio(accounts_receivables, total_sales):
    return accounts_receivables / total_sales if total_sales else None

def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
    return allowance_for_doubtful_accounts / accounts_receivables if accounts_receivables else None

def asset_to_debt_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def current_ratio(current_assets, current_liabilities):
    return current_assets / current_liabilities if current_liabilities else None

def quick_ratio(current_assets, inventory, current_liabilities):
    return (current_assets - inventory) / current_liabilities if current_liabilities else None

def cash_ratio(cash_and_cash_equivalents, current_liabilities):
    return cash_and_cash_equivalents / current_liabilities if current_liabilities else None

def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
    return non_current_assets / non_current_liabilities if non_current_liabilities else None

def debt_to_equity_ratio(total_liabilities, total_equity):
    return total_liabilities / total_equity if total_equity else None

def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
    return non_current_liabilities / equity if equity else None

def time_interest_earned(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
    return total_liabilities / (equity - intangible_assets) if (equity - intangible_assets) else None

ratio_functions = {
    'receivables_to_payables_ratio': [['BS_130', 'BS_210'], 'BS_300'],  # accounts_receivable, total_liabilities
    'receivables_to_total_assets_ratio': [['BS_130', 'BS_210'], 'BS_270'],  # accounts_receivable, total_assets
    'debt_to_total_capital_ratio': ['BS_300', 'BS_440'],  # total_liabilities, total_capital
    'receivables_to_sales_ratio': [['BS_131', 'BS_211'], 'IS_010'],  # accounts_receivables, total_sales
    'allowance_for_doubtful_accounts_ratio': [['BS_137', 'BS_219'], ['BS_131', 'BS_211']],  # allowance_for_doubtful_accounts, accounts_receivables
    'asset_to_debt_ratio': ['BS_270', 'BS_300'],  # total_assets, total_liabilities
    'current_ratio': [['BS_100', 'BS_151'], 'BS_310'],  # current_assets_for_liquidity, current_liabilities
    'quick_ratio': [['BS_100', 'BS_151'], 'BS_140', 'BS_310'],  # current_assets - inventory, current_liabilities
    'cash_ratio': ['BS_110', 'BS_310'],  # cash_and_cash_equivalents, current_liabilities
    'long_term_debt_coverage_ratio': ['BS_200', 'BS_330'],  # non_current_assets, non_current_liabilities
    'debt_to_equity_ratio': ['BS_300', 'BS_400'],  # total_liabilities, total_equity
    'long_term_debt_to_equity_capital_ratio': ['BS_330', 'BS_400'],  # long_term_liabilities, equity
    'time_interest_earned': [['IS_050', 'IS_023'], 'IS_023'],  # EBIT, interest_expense
    'debt_to_tangible_net_worth_ratio': ['BS_300', 'BS_400', 'BS_227'],  # total_liabilities, equity, intangible_assets
}

# Create a DataFrame to store the results
results_2 = []

# Iterate through pivoted data
for index, row in pivot_df_2.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        input_values = []
        if ratio == 'current_ratio':
            # Subtract BS_151 from BS_100 for current ratio calculation
            current_assets = row[inputs[0][0]] - row[inputs[0][1]] if inputs[0][0] in row.index and inputs[0][1] in row.index else None
            input_values.append(current_assets)
            input_values.append(row[inputs[1]] if inputs[1] in row.index else None)
        elif ratio == 'quick_ratio':
            # Subtract BS_140 from BS_100 for quick ratio calculation
            current_assets = row[inputs[0][0]] - row[inputs[0][1]] if inputs[0][0] in row.index and inputs[0][1] in row.index else None
            inventory = row[inputs[1]] if inputs[1] in row.index else None
            current_liabilities = row[inputs[2]] if inputs[2] in row.index else None
            input_values.extend([current_assets, inventory, current_liabilities])
        else:
            # For all other ratios, sum values if needed or fetch directly
            for input_name in inputs:
                if isinstance(input_name, list):
                    value_sum = sum([row[i] for i in input_name if i in row.index])
                    input_values.append(value_sum)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_2.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_2 = pd.DataFrame(results_2)

# Display or export the DataFrame
ratios_df_2


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,receivables_to_payables_ratio,0.165393
1,BCM,2021,0,receivables_to_total_assets_ratio,0.107466
2,BCM,2021,0,debt_to_total_capital_ratio,0.649764
3,BCM,2021,0,receivables_to_sales_ratio,0.601035
4,BCM,2021,0,allowance_for_doubtful_accounts_ratio,-0.013684
...,...,...,...,...,...
3033,VRE,2024,2,long_term_debt_coverage_ratio,5.792414
3034,VRE,2024,2,debt_to_equity_ratio,0.310486
3035,VRE,2024,2,long_term_debt_to_equity_capital_ratio,0.172701
3036,VRE,2024,2,time_interest_earned,10.235884


## Phân tích rủi ro tài chính (Financial risk)

In [10]:
# # financial_leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
# # 0< financial_leverage < 1 : Close to 0 is better
# def financial_leverage(total_liabilities, total_lia_and_equity):
#     return total_liabilities / total_lia_and_equity

# # Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219)/ total_assets (BS_270)
# def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
#     return allowance_for_doubtful_accounts / total_assets

# # Hệ số tài trợ thường xuyên (Permanent Financing Ratio) = permanent_capital(BS_400+BS_330) / total_lia_and_equity (BS_440)
# def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
#     return permanent_capital / total_lia_and_equity





In [11]:
import pandas as pd

# Create a new pivot table for the new ratios
pivot_df_3 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
def financial_leverage(total_liabilities, total_lia_and_equity):
    return total_liabilities / total_lia_and_equity if total_lia_and_equity else None

# Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219) / total_assets (BS_270)
def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
    return allowance_for_doubtful_accounts / total_assets if total_assets else None

# Permanent Financing Ratio (Hệ số tài trợ thường xuyên) = permanent_capital(BS_400 + BS_330) / total_lia_and_equity (BS_440)
def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
    return permanent_capital / total_lia_and_equity if total_lia_and_equity else None

new_ratio_functions = {
    'financial_leverage': ['BS_300', 'BS_440'],  # total_liabilities (BS_300), total_lia_and_equity (BS_440)
    'allowance_for_doubtful_accounts_to_total_assets_ratio': [['BS_137', 'BS_219'], 'BS_270'],  # allowance_for_doubtful_accounts (BS_137+BS_219), total_assets (BS_270)
    'permanent_financing_ratio': [['BS_400', 'BS_330'], 'BS_440'],  # permanent_capital (BS_400 + BS_330), total_lia_and_equity (BS_440)
}

# Create a DataFrame to store the results of the new ratios
results_3 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_3.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_functions.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like permanent capital or allowance for doubtful accounts
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_3.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_3 = pd.DataFrame(results_3)

# Display or export the DataFrame
ratios_df_3


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,financial_leverage,0.649764
1,BCM,2021,0,allowance_for_doubtful_accounts_to_total_asset...,-0.001174
2,BCM,2021,0,permanent_financing_ratio,0.622688
3,BCM,2021,3,financial_leverage,0.661998
4,BCM,2021,3,allowance_for_doubtful_accounts_to_total_asset...,-0.000962
...,...,...,...,...,...
646,VRE,2024,1,allowance_for_doubtful_accounts_to_total_asset...,-0.005578
647,VRE,2024,1,permanent_financing_ratio,0.864835
648,VRE,2024,2,financial_leverage,0.236925
649,VRE,2024,2,allowance_for_doubtful_accounts_to_total_asset...,-0.006455


## phân tích kết quả kinh doanh 

In [12]:
# # Doanh thu thuần về bán hàng và cung cấp dịch vụ ,IS_010
# # Doanh thu hoạt động tài chính,IS_021,Financial income
# # doanh thu thuần = IS_010 + IS_021

# # Financial Income to Net Revenue Ratio = Financial Income(IS_021)/ Net Revenue
# def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
#     return financial_income / net_revenue



In [13]:
import pandas as pd

# Create a new pivot table for the Financial Income to Net Revenue Ratio
pivot_df_4 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Income to Net Revenue Ratio = Financial Income(IS_021) / Net Revenue
def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
    return financial_income / net_revenue if net_revenue else None

# Dictionary to hold the category codes for the new ratio
new_ratio_function = {
    'financial_income_to_net_revenue_ratio': ['IS_021', 'IS_010']  # financial_income (IS_021), net_revenue (IS_010)
}

# Create a DataFrame to store the results of the new ratio
results_4 = []

# Iterate through the pivot table to calculate the new ratio
for index, row in pivot_df_4.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_function.items():
        input_values = []
        for input_name in inputs:
            input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_4.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_4 = pd.DataFrame(results_4)

# Display or export the DataFrame
ratios_df_4


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,financial_income_to_net_revenue_ratio,0.009825
1,BCM,2021,3,financial_income_to_net_revenue_ratio,0.017711
2,BCM,2021,4,financial_income_to_net_revenue_ratio,0.004083
3,BCM,2022,0,financial_income_to_net_revenue_ratio,0.011301
4,BCM,2022,1,financial_income_to_net_revenue_ratio,0.015663
...,...,...,...,...,...
212,VRE,2023,2,financial_income_to_net_revenue_ratio,0.128014
213,VRE,2023,3,financial_income_to_net_revenue_ratio,0.090443
214,VRE,2023,4,financial_income_to_net_revenue_ratio,0.146398
215,VRE,2024,1,financial_income_to_net_revenue_ratio,0.193004


## phân tích khả năng sinh lời ( profitability )

In [14]:
# # ROA : Return on Assets = net_income(IS_060) / total_assets
# def return_on_assets(net_income, total_assets):
#     return net_income / total_assets

# #Return on fixed assets = net_income(IS_060) / average_fixed_assets (BS_220)
# def return_on_fixed_assets(net_income, average_fixed_assets):
#     return net_income / average_fixed_assets

# # #RNOA= Net Operating Profit After Tax (NOPAT)/Average Net Operating Assets
# # def RNOA(NOPAT, average_net_operating_assets):
# #     return NOPAT / average_net_operating_assets

# # return on long-term operating assets = net_income(IS_060) / average_long_term_operating_assets
# def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
#     return net_income / average_long_term_operating_assets

# # SỨC SINH LỢI CƠ BẢN CỦA TÀI SẢN : BEPR (Basic Earning Power Ratio) = EBIT(IS_050 +IS_023) / total_assets
# def BEPR(EBIT, total_assets):
#     return EBIT / total_assets

# #ROE = Net Income/  Equity
# def ROE(net_income, equity):
#     return net_income / equity

# # Return on Common Equity (sức sinh lời của vốn cổ phần thường) = Net Income (IS_060) - (Preferred Dividends-CF-036)/ Average Common Equity
# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return net_income - preferred_dividends / average_common_equity

# # Profitability of Cost of Goods Sold (COGS) = net income from operating(IS_030)/ COGS (IS_011)
# def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
#     return net_income_from_operating / COGS

# # Hệ số chênh lệch giá (Price Spread Ratio): gross_profit(IS_020) / COGS(IS_011)
# def price_spread_ratio(gross_profit, COGS):
#     return gross_profit / COGS

# # Sức sinh lợi của chi phí kinh doanh (Profitability of Operating Expenses): net_income_from_operating(IS_030) / total_operating_expenses(IS_025+IS_026+IS_011)
# def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
#     return net_income_from_operating / total_operating_expenses

# # ROS = Net Income/ Net Sales (IS_010+IS_021)
# def ROS(net_income, net_sales):
#     return net_income / net_sales

# # adjusted ROS = net_income(IS_060) - cổ tức ưu đãi/ net_sales(IS_010+IS_021) 

# # Operating Profit Margin (OPM) = Net profit from operating activities (IS_030)/ Net Sales
# def operating_profit_margin(net_profit_from_operating, net_sales):
#     return net_profit_from_operating / net_sales

# # Gross Profit Margin (GPM) = Gross Profit(IS_020)/ Net Sales
# def gross_profit_margin(gross_profit, net_sales):
#     return gross_profit / net_sales

# #net operating profit margin = lợi nhuận sau thuế từ hđkd(net operating profit after tax, NOPAT)/ net sales(IS_010)
# # NOPAT = lợi nhuận trc thuế từ hđkd x (1 - thuế suất)
# def net_operating_profit_margin(NOPAT, net_sales):
#     return NOPAT / net_sales

# # price to book ratio = giá cổ phiếu / giá trị sổ sách ( market price per share/book value per share)
# # price to earnings ratio = giá cổ phiếu / lợi nhuận trên cổ phiếu
# #Dividend Payout Ratio =  Dividend per Share/ Earnings per Share
# # Dividend Yield = Dividend per Share/ Market Price per Share





In [15]:
import pandas as pd

# Create a new pivot table for the ratios in pivot_df_5
pivot_df_5 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Ratio calculation functions
def return_on_assets(net_income, total_assets):
    return net_income / total_assets if total_assets else None

def return_on_fixed_assets(net_income, average_fixed_assets):
    return net_income / average_fixed_assets if average_fixed_assets else None

def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
    return net_income / average_long_term_operating_assets if average_long_term_operating_assets else None

def Basic_Earning_Power_Ratio(EBIT, total_assets):
    return EBIT / total_assets if total_assets else None

def Return_on_equity(net_income, equity):
    return net_income / equity if equity else None

# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return (net_income - preferred_dividends) / average_common_equity if average_common_equity else None

def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
    return net_income_from_operating / COGS if COGS else None

def price_spread_ratio(gross_profit, COGS):
    return gross_profit / COGS if COGS else None

def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
    return net_income_from_operating / total_operating_expenses if total_operating_expenses else None

def Return_on_sales(net_income, net_sales):
    return net_income / net_sales if net_sales else None

def operating_profit_margin(net_profit_from_operating, net_sales):
    return net_profit_from_operating / net_sales if net_sales else None

def gross_profit_margin(gross_profit, net_sales):
    return gross_profit / net_sales if net_sales else None

def net_profit_margin(net_income, net_sales):
    return Return_on_sales(net_income, net_sales)

# Dictionary to hold functions and corresponding category codes
ratio_functions_5 = {
    'return_on_assets': ['IS_060', 'BS_270'],  # net_income (IS_060), total_assets (BS_270)
    'return_on_fixed_assets': ['IS_060', 'BS_220'],  # net_income (IS_060), average_fixed_assets (BS_220)
    'return_on_long_term_operating_assets': ['IS_060', 'BS_240'],  # net_income (IS_060), average_long_term_operating_assets (BS_240)
    'Basic_Earning_Power_Ratio': [['IS_050', 'IS_023'], 'BS_270'],  # EBIT (IS_050 + IS_023), total_assets (BS_270)
    'Return_on_equity': ['IS_060', 'BS_400'],  # net_income (IS_060), equity (BS_400)
    # 'return_on_common_equity': ['IS_060', 'CF_036', 'BS_400'],  # net_income (IS_060), preferred_dividends (CF_036), average_common_equity (BS_400)
    'profitability_of_cost_of_goods_sold': ['IS_030', 'IS_011'],  # net_income_from_operating (IS_030), COGS (IS_011)
    'price_spread_ratio': ['IS_020', 'IS_011'],  # gross_profit (IS_020), COGS (IS_011)
    'profitability_of_operating_expenses': ['IS_030', ['IS_025', 'IS_026', 'IS_011']],  # net_income_from_operating (IS_030), total_operating_expenses (IS_025 + IS_026 + IS_011)
    'Return_on_sales': ['IS_060', ['IS_010','IS_021']],  # net_income (IS_060), net_sales (IS_010)
    'operating_profit_margin': ['IS_030',  ['IS_010','IS_021']],  # NOPAT, net_sales (IS_010)
    'gross_profit_margin': ['IS_020',  ['IS_010','IS_021']],  # gross_profit (IS_020), net_sales (IS_010)
    'net_profit_margin': ['IS_060',  ['IS_010','IS_021']]  # net_income (IS_060), net_sales (IS_010)
}

# Create a DataFrame to store the results of the new ratios
results_5 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_5.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions_5.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like EBIT or total operating expenses
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_5.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_5 = pd.DataFrame(results_5)

# Display or export the DataFrame
ratios_df_5


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,return_on_assets,0.029769
1,BCM,2021,0,return_on_fixed_assets,1.194759
2,BCM,2021,0,return_on_long_term_operating_assets,0.634548
3,BCM,2021,0,Basic_Earning_Power_Ratio,0.051324
4,BCM,2021,0,Return_on_equity,0.084997
...,...,...,...,...,...
2599,VRE,2024,2,profitability_of_operating_expenses,0.831977
2600,VRE,2024,2,Return_on_sales,0.352380
2601,VRE,2024,2,operating_profit_margin,0.426134
2602,VRE,2024,2,gross_profit_margin,0.417220


## phân tích cashflow


In [18]:
# # Earnings Before Interest, Taxes, Depreciation and Amortization (EBITDA) = EBIT(IS_050) + Depreciation and Amortization (CF_002)
# def EBITDA(EBIT, depreciation_and_amortization):
#     return EBIT + depreciation_and_amortization
# # Cash Flow Adequacy Ratio = Cash Flow from Operations/ Capital_Expenditures+Debt_Repayments+Dividends_Paid
# # Free Cash Flow (FCF)=Operating_Net_Cash_Flow (CF_020) − Capital_Expenditures_(CAPEX) ( CF_021+CF_023) - Chi trả cổ tức (CF_036)
# def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
#     return operating_net_cash_flow - capital_expenditures - dividends_paid

# # free cash flow to the operating cash flow ratio = free_cash_flow / operating_net_cash_flow (CF_020)
# def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
#     return free_cash_flow / operating_net_cash_flow

# # cash debt coverage ratio = operating_net_cash_flow(CF_020)/ avg_total_liabilities
# def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
#     return operating_net_cash_flow / avg_total_liabilities

# #Hệ số đảm bảo khả năng chi trả lãi vay: cash interest covarage(CIC)= operating_net_cash_flow(CF_020) + interest_expense(IS_023) / interest_expense(IS_023)
# def cash_interest_coverage(operating_net_cash_flow, interest_expense):
#     return operating_net_cash_flow + interest_expense / interest_expense

# # Hệ số khả năng chi trả cổ tức : Dividend payment ratio = perating_net_cash_flow(CF_020)/ dividends_payables (TMBCTC)

# # cash return on assets = operating_net_cash_flow(CF_020) / avg_total_assets
# def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
#     return operating_net_cash_flow / avg_total_assets

# #cash return on fixed assets = operating_net_cash_flow(CF_020) / avg_fixed_assets
# def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
#     return operating_net_cash_flow / avg_fixed_assets

# # CFO to total equity = operating_net_cash_flow(CF_020) / avg_total_equity
# def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
#     return operating_net_cash_flow / avg_total_equity

# # cash flow return on investment = Net_Cash_Flow_before_Tax_and_Interest_Payments(CF_050 +CF_014 + CF_015) / VỐN SỬ DỤNG BÌNH QUÂN (BS_400+BS_330)

# # Vốn cổ phần phổ thông(Common Equity) = BS_411 - BS_415
# # Cash return on shareholders = operating_net_cash_flow(CF_020)- cổ tức ưu đãi/ số cổ phiếu phổ thông bình quân trong kỳ
# # cas flow from sales to sales = operating_net_cash_flow(CF_020)- CF_036/ net_operating_sales(IS_010)
# def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_operating_sales):
#     return operating_net_cash_flow / net_operating_sales

# # cash flow margin = operating_net_cash_flow(CF_020) / tổng doanh thu thuần (IS_010+ IS_021)
# def cash_flow_margin(operating_net_cash_flow, total_revenue):
#     return operating_net_cash_flow / total_revenue

# # Earning quality ratio = operating_net_cash_flow(CF_020) / net_income(IS_060)
# def earning_quality_ratio(operating_net_cash_flow, net_income):
#     return operating_net_cash_flow / net_income
# # free cash flows to the firm (FCFF) = EBIT (IS_050 +IS_023). (1 - tax rate) + Depreciation  - Capital Expenditures - Change in Net Working Capital 


In [19]:
import pandas as pd

# Create a new pivot table for the cash flow ratios in pivot_df_7
pivot_df_6 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Function to get the value of quarter 0 from the previous year for a given stock_code
def get_previous_year_q0_value(stock_code, year, category_code):
    try:
        return pivot_df_6.loc[(stock_code, year - 1, 0), category_code]
    except KeyError:
        return None

# Ratio calculation functions
def EBITDA(EBIT, depreciation_and_amortization):
    return EBIT + depreciation_and_amortization

def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
    return operating_net_cash_flow - capital_expenditures - dividends_paid

def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
    return free_cash_flow / operating_net_cash_flow if operating_net_cash_flow else None

def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
    return operating_net_cash_flow / avg_total_liabilities if avg_total_liabilities else None

def cash_interest_coverage(operating_net_cash_flow, interest_expense):
    return (operating_net_cash_flow + interest_expense) / interest_expense if interest_expense else None

def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
    return operating_net_cash_flow / avg_total_assets if avg_total_assets else None

def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
    return operating_net_cash_flow / avg_fixed_assets if avg_fixed_assets else None

def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
    return operating_net_cash_flow / avg_total_equity if avg_total_equity else None

def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_operating_sales):
    return operating_net_cash_flow / net_operating_sales if net_operating_sales else None

def cash_flow_margin(operating_net_cash_flow, total_revenue):
    return operating_net_cash_flow / total_revenue if total_revenue else None

def earning_quality_ratio(operating_net_cash_flow, net_income):
    return operating_net_cash_flow / net_income if net_income else None

# Dictionary to map ratios to category codes (with average calculations for avg fields)
cash_flow_ratio_functions_6 = {
    'EBITDA': ['IS_050', 'CF_002'],  # EBIT (IS_050), depreciation_and_amortization (CF_002)
    'free_cash_flow': ['CF_020', ['CF_021', 'CF_023'], 'CF_036'],  # operating_net_cash_flow (CF_020), capital_expenditures (CF_021 + CF_023), dividends_paid (CF_036)
    'free_cash_flow_to_operating_cash_flow_ratio': ['free_cash_flow', 'CF_020'],  # free_cash_flow, operating_net_cash_flow (CF_020)
    'cash_debt_coverage_ratio': ['CF_020', 'BS_300'],  # operating_net_cash_flow (CF_020), avg_total_liabilities (BS_300)
    'cash_interest_coverage': ['CF_020', 'IS_023'],  # operating_net_cash_flow (CF_020), interest_expense (IS_023)
    'cash_return_on_assets': ['CF_020', 'BS_270'],  # operating_net_cash_flow (CF_020), avg_total_assets (BS_270)
    'cash_return_on_fixed_assets': ['CF_020', 'BS_220'],  # operating_net_cash_flow (CF_020), avg_fixed_assets (BS_220)
    'CFO_to_total_equity': ['CF_020', 'BS_400'],  # operating_net_cash_flow (CF_020), avg_total_equity (BS_400)
    'cash_flow_from_sales_to_sales': ['CF_020', 'IS_010'],  # operating_net_cash_flow (CF_020), net_sales (IS_010+IS_021)
    'cash_flow_margin': ['CF_020', ['IS_010', 'IS_021']],  # operating_net_cash_flow (CF_020), total_revenue (IS_010 + IS_021)
    'earning_quality_ratio': ['CF_020', 'IS_060'],  # operating_net_cash_flow (CF_020), net_income (IS_060)
}

# Create a DataFrame to store the results of the new cash flow ratios
cash_flow_results_6 = []

# Iterate through the pivot table to calculate the cash flow ratios
for index, row in pivot_df_6.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in cash_flow_ratio_functions_6.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like capital_expenditures or total_revenue
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                if input_name in ['BS_300', 'BS_270', 'BS_220', 'BS_400']:  
                    prev_q0_value = get_previous_year_q0_value(stock_code, year, input_name)
                    current_value = row[input_name] if input_name in row.index else None
                    if current_value is not None and prev_q0_value is not None:
                        avg_value = (current_value + prev_q0_value) / 2
                        input_values.append(avg_value)
                    else:
                        input_values.append(None)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            cash_flow_results_6.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_6 = pd.DataFrame(cash_flow_results_6)

# Display or export the DataFrame
ratios_df_6


,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,EBITDA,1.929737e+06
1,BCM,2021,0,free_cash_flow,1.458006e+06
2,BCM,2021,0,cash_interest_coverage,1.445778e+00
3,BCM,2021,0,cash_flow_from_sales_to_sales,4.902423e-02
4,BCM,2021,0,cash_flow_margin,4.854725e-02
...,...,...,...,...,...
1989,VRE,2024,2,cash_return_on_fixed_assets,-3.422275e+00
1990,VRE,2024,2,CFO_to_total_equity,-2.854306e-02
1991,VRE,2024,2,cash_flow_from_sales_to_sales,-4.477198e-01
1992,VRE,2024,2,cash_flow_margin,-3.828234e-01


In [20]:
VNM_cash_debt_coverage_ratio_q1_2024 = ratios_df_6[
    (ratios_df_6['stock_code'] == 'VNM') & 
    (ratios_df_6['year'] == 2024) & 
    (ratios_df_6['quarter'] == 1) & 
    (ratios_df_6['ratio_code'] == 'cash_debt_coverage_ratio')
]['data'].values[0]

print(f"Cash Debt Coverage Ratio of VNM in Q1 2024: {VNM_cash_debt_coverage_ratio_q1_2024}")

Cash Debt Coverage Ratio of VNM in Q1 2024: 0.05428913989289794


In [21]:
# Concatenate all the DataFrames
ratios_df = pd.concat([ratios_df_1, ratios_df_2, ratios_df_3, ratios_df_4, ratios_df_5, ratios_df_6], ignore_index=True)

# Display the concatenated DataFrame
ratios_df

,stock_code,year,quarter,ratio_code,data
0,BCM,2021,0,equity_ratio,0.350236
1,BCM,2021,0,long_term_asset_self_financing_ratio,1.539085
2,BCM,2021,0,fixed_asset_self_financing_ratio,8.668917
3,BCM,2021,0,general_solvency_ratio,1.539021
4,BCM,2021,0,return_on_investment,0.029769
...,...,...,...,...,...
11103,VRE,2024,2,cash_return_on_fixed_assets,-3.422275
11104,VRE,2024,2,CFO_to_total_equity,-0.028543
11105,VRE,2024,2,cash_flow_from_sales_to_sales,-0.447720
11106,VRE,2024,2,cash_flow_margin,-0.382823


In [39]:
ratios_df.rename(columns={'ratio_code': 'function_name'}, inplace=True)

In [48]:
map_df = pd.read_csv('csv/map_ratio_code.csv')
map_df['function_name'] = map_df['function_name'].str.strip()
map_df

,ratio_name,ratio_code,function_name
0,Equity Ratio,ER,equity_ratio
1,Long Term Asset Self Financing Ratio,LTASFR,long_term_asset_self_financing_ratio
2,Fixed Asset Self Financing Ratio,FASFR,fixed_asset_self_financing_ratio
3,General Solvency Ratio,GSR,general_solvency_ratio
4,Return On Investment,ROI,return_on_investment
5,Basic Earning Power,BEP,basic_earning_power
6,Debt To Assets Ratio,DTAR,debt_to_assets_ratio
7,Debt To Equity Ratio,DTER,debt_to_equity_ratio
8,Short Term Debt To Assets Ratio,STDTAR,short_term_debt_to_assets_ratio
9,Interest Coverage Ratio,ICR,interest_coverage_ratio


In [45]:
ratios_df_2 = pd.merge(ratios_df, map_df, on='function_name', how='left')
ratios_df_2

,stock_code,year,quarter,function_name,data,ratio_name,ratio_code
0,BCM,2021,0,equity_ratio,0.350236,Equity Ratio,ER
1,BCM,2021,0,long_term_asset_self_financing_ratio,1.539085,Long Term Asset Self Financing Ratio,LTASFR
2,BCM,2021,0,fixed_asset_self_financing_ratio,8.668917,Fixed Asset Self Financing Ratio,FASFR
3,BCM,2021,0,general_solvency_ratio,1.539021,General Solvency Ratio,GSR
4,BCM,2021,0,return_on_investment,0.029769,Return On Investment,ROI
...,...,...,...,...,...,...,...
11103,VRE,2024,2,cash_return_on_fixed_assets,-3.422275,Cash Return On Fixed Assets,CROFA
11104,VRE,2024,2,CFO_to_total_equity,-0.028543,CFO To Total Equity,CFOTTE
11105,VRE,2024,2,cash_flow_from_sales_to_sales,-0.447720,Cash Flow From Sales To Sales,CFFSTS
11106,VRE,2024,2,cash_flow_margin,-0.382823,Cash Flow Margin,CFM


In [47]:
ratios_df_2[ratios_df_2['function_name'].isnull()]

,stock_code,year,quarter,function_name,data,ratio_name,ratio_code
